
### Project Overview
    Document parameter explorations of the 1D Fiete (2009) grid-cell model, integration of hippocampal place-cell feedback (modified from V-H), and single-pattern learning with attractor-based denoising analysis across varied Gaussian noise levels, highlighting the extended convergence times required as noise magnitude increases.
### Content
 @2026-1-31
+ 调节Fiete,2009的参数，改变bump的数量
+ 加入海马模块到grid中
+ 单一模式学习与吸引子验证分析
    + 加入不同方差水平的高斯噪声吗，网络迭代演化
    + @2026/2/2 随着噪声的增加，网络需要更长时间演化到训练后的吸引子状态      
+ *这个代码没有提取相位和多模式学习的内容，只做到学习单个模式*


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation
from scipy.stats import poisson
from datetime import datetime


## Parameters

### Grid

In [ ]:
N = 64   # grid cell number
# Network parameters
m = 5  # CV = 1/sqrt(m)
tau_s = 30/1000    # synaptic time constant (s)
# 偏好位置分布在0-1之间
x_prefs = np.arange(1, N+1).reshape(-1, 1) / N  # Inherited location preferences
# FeedForward input
e_mu = 1.5     # velocity gain
beta_0 = 70        # uniform input
g_gp = 1.0  # gain for place -> grid projections
# Graphing parameters
bins = np.linspace(0 + 0.01, 1 - 0.01, 50)

alpha = 1000       # weight imbalance parameter
alpha_bar = 1.001    # Center Surround weight params
gamma = 1.05/100   # Center Surround weight params
beta_w = 0.80/100     # Center Surround weight params (renamed to avoid conflict with beta_vel)
sigma1 = 1
sigma2 = 1
# x_range = 32    # 直接控制权重函数的x范围
# Relative indices for weight function
z = np.arange(-N/2, N/2, 1) # [-32,31)  # x从-N/2 - N/2
# z = np.linspace(-x_range, x_range, N)  # 这种方法不好控制中点为0
# Weight setup
crossSection = alpha * (alpha_bar*np.exp(-gamma * z**2/sigma1**2) - np.exp(-beta_w * z**2/sigma2**2))

crossSection_shift = np.roll(crossSection, int(N/2))
envelope = np.ones((N, 1)) # periodic topology

#### visualize kernel

In [ ]:
# Plot weight functions before and after shift
# fig, ax = plt.subplots(1,2,figsize = (12,5))
# ax[0].plot(crossSection)
# ax[0].set_title('before shift')
# ax[1].plot(crossSection_shift)
# ax[1].set_title('after shift')
# plt.show()

In [ ]:
# Initialize weight matrices
# MATLAB: zeros(N, N)
W_RR = np.zeros((N, N))
W_LL = np.zeros((N, N))
W_RL = np.zeros((N, N))
W_LR = np.zeros((N, N))
 # Construct weight matrices
for i in range(N):
    # MATLAB uses 1-based indexing, Python uses 0-based
    # In MATLAB: circshift(crossSection, [0 i - 1])
    # In Python: np.roll(crossSection, i - 1)
    W_RR[i, :] = np.roll(crossSection_shift, i - 1)  # Right neurons to Right neurons
    W_LL[i, :] = np.roll(crossSection_shift, i + 1)  # Left neurons to Left neurons
    W_RL[i, :] = np.roll(crossSection_shift, i + 1)  # Left neurons to Right neurons
    W_LR[i, :] = np.roll(crossSection_shift, i - 1)  # Right neurons to Left neurons

### HPC
设置海马模块的初始参数

In [ ]:
# Hippocampal (place cell) module parameters
Nh = 100  # number of place cells
tau_h = 300 / 1000  # hippocampal time constant (s)
theta = 0.2  # activation threshold
hebb_lr = 1e-4  # Hebbian learning rate for place->grid synapses
epsilon = 1e-9
relu = lambda x: np.maximum(x, 0.0)
gain_pg = 1.0  # gain for grid -> place projections
Iext2 = 0.0  # external input to place cells
# Synaptic weights
W_pg = np.random.normal(0.0, 1.0, size=(Nh, 2 * N))  # grid -> place
# Create sparse connectivity mask for grid -> place projections
r_pg = 0.1  # sparsity ratio: retain 10% of connections
mask_pg = np.random.rand(Nh, 2 * N) < r_pg
W_pg = W_pg * mask_pg

# 初始从p->g的投射为0
Wgp_L = np.zeros((N, Nh))  # place -> left grid
Wgp_R = np.zeros((N, Nh))  # place -> right grid
p_prev = np.zeros((Nh, 1))  # initial place-cell activity


## One-Pattern Learning

### 1. Initializing: Grid only Simulation

Generate Positional Trajectories

In [ ]:
# Temporal parameters
T = 20            # length of integration time blocks (s)
dt = 1/2000        # step size of numerical integration (s)
# Trajectory Data (Sinusoidal)
# MATLAB: dt:dt:T creates a range from dt to T with step dt
time_steps = np.arange(dt, T + dt, dt)
x = (np.sin(time_steps * 2 * np.pi / 10) + 1) / 2  # generate sine wave
# x = np.hstack([1/1000*np.arange(1000), np.zeros(len(time_steps)-1000)])  # flat trajectory
# Calculate velocity
v = np.zeros_like(x)
v[1:] = (x[1:] - x[:-1]) / dt  # more efficient than loop

在加入海马模块前，先跑grid的部分，生成grid的初始状态

In [ ]:
# Fiete,2009
n_steps = int(T/dt)
# record population spikes
r_all = np.zeros((2*N, n_steps))
# calculate spike counts in each time step
s_prev = np.zeros((2*N, 1))      # Population activity (256,1)
spk = np.zeros((2*N, len(time_steps)))  # Total spiking
spk_count = np.zeros((2*N, 1))   # Current spiking
# v = np.zeros_like(x)
for t in range(1, n_steps):
    
    # two population of neurons
    # left population
    v_L = (1-e_mu * v[t])
    g_LL = W_LL @ s_prev[:N]
    g_LR = W_LR @ s_prev[N:]
    G_L = v_L * ((g_LL + g_LR) + envelope * beta_0)
    # RIGHT population
    v_R = (1 + e_mu * v[t])
    g_RR = W_RR @ s_prev[N:]  # R->R
    g_RL = W_RL @ s_prev[:N]  # L->R
    
    G_R = v_R * ((g_RR + g_RL) + envelope * beta_0)
    
    G = np.vstack([G_L, G_R])
        
    # Linear transfer function (ReLU) + Poisson Spike
    F = G * (G >= 0)  # ReLU activation

    # F_expanded = np.tile(F, (1, m)) * dt
    # spk_sub = poisson.rvs(F_expanded)  # Poisson random numbers
    # spk_count += np.sum(spk_sub, axis=1, keepdims=True)   # 在m个时间步长中累计发放数量
    # # Determine actual spikes for this time step
    # spk[:, t] = np.floor(spk_count.flatten() / m) # 
    # spk_count = spk_count % m  # Keep remainder 跨时间步骤累计参与的spike量
    spk[:, t:t+1] = F * dt  # Ensure expected spike count matches firing rate
    # Update population activity
    s_new = s_prev + spk[:, t:t+1] - s_prev * dt / tau_s
    s_prev = s_new
    r_all[:,t] = s_new.flatten()

### Visualization

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(figsize=(10,6))
# Initialize lines
line_left, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'r-', alpha=0.7, label='Left Population')
line_right, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'b-', alpha=0.7, label='Right Population')
line_combined, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'g-', linewidth=2, label='Combined')
# Add position indicator
pos_line = ax.axvline(0, color='k', linestyle='--', alpha=0.5, label='Current Position')
ax.set_xlim(0, 1)
ax.set_ylim(0, np.max(r_all) * 2.05)
ax.set_xlabel('Position Preference')
ax.set_ylabel('Neural Activity')
ax.set_title('1D grid Fiete 2009')
ax.legend()
ax.grid(True, alpha=0.3)
def update(frame):
    t_idx = frame
    
    # Update activity lines
    left_activity = r_all[:N, t_idx]
    right_activity = r_all[N:, t_idx]
    combined_activity = left_activity + right_activity
    
    line_left.set_ydata(left_activity)
    line_right.set_ydata(right_activity)
    line_combined.set_ydata(combined_activity)
    
    # Update position line
    current_pos = x[t_idx]
    pos_line.set_xdata([current_pos, current_pos])
    

    return line_left, line_right, line_combined, pos_line

# Create animation
frame_step = 20
ani = FuncAnimation(fig, update, frames=range(0, min(n_steps, 20000), frame_step),  # Limit to 1000 frames for performance
                    interval=50, blit=True, repeat=True)
plt.show()
plt.tight_layout()


In [ ]:
ani.event_source.stop()
%matplotlib inline

In [ ]:
print("Saving animation...")
folder = 'visualization'
cur_dir = os.getcwd()
folder_path = os.path.join(cur_dir, folder)
print(folder_path)
if not os.path.exists(folder_path):
    print(f'文件夹{folder}生成')
    os.makedirs(folder_path)
timestamp = datetime.now().strftime("%m%d_%H%M%S")
# ani.save('neural_sheet_activity'+timestamp+'.mp4', writer='ffmpeg', fps=20, dpi=100)
ani.save(os.path.join(folder_path, f'grid32_lower_beta-w_{N}'+timestamp+'.gif'), writer='pillow', fps=20, dpi=100)
print("Animation saved")

### Get Grid Activity Patterns

In [ ]:
gL = r_all[:N,-1]
gR = r_all[N:2*N,-1]
g = np.mean(np.vstack([gL, gR]), axis=0)
plt.title('Initial grid pattern')
plt.plot(g)
plt.show()

### 2.  Training: Loop Simulation

1. 实时学习： 从grid初始状态出发，进行动力学迭代

In [ ]:
# Fiete,2009 with hippocampal feedback
T2 = 3
n_steps = int(T2/dt)

v = np.zeros(int(T2/dt))   # 速度输入为0
# record population spikes
r_all = np.zeros((2*N, n_steps))
r_place = np.zeros((Nh, n_steps))

# calculate spike counts in each time step
# s_prev = np.zeros((2*N, 1))      # Population activity (2N,1)
s_prev = np.hstack([g,g]).T.reshape(-1,1)     # initialize with previous g
p_prev = np.zeros((Nh, 1))        # Place-cell population
spk = np.zeros((2*N, n_steps))  # Total spiking
spk_count = np.zeros((2*N, 1))   # Current spiking
# v = np.zeros_like(x)
for t in range(1, n_steps):
    
    # Hippocampal (place-cell) dynamics
    hippo_drive = gain_pg * W_pg @ s_prev + Iext2    # grid -> place input
    phi_term = relu(hippo_drive - theta).reshape(-1,1)
    p_new = p_prev + dt / tau_h * (-p_prev + phi_term)
    r_place[:, t:t+1] = p_new
    
    # two population of neurons
    # left population
    v_L = (1-e_mu * v[t])
    g_LL = W_LL @ s_prev[:N]
    g_LR = W_LR @ s_prev[N:]
    G_L = v_L * ((g_LL + g_LR) + envelope * beta_0) + g_gp * Wgp_L @ p_new
    # RIGHT population
    v_R = (1 + e_mu * v[t])
    g_RR = W_RR @ s_prev[N:]  # R->R
    g_RL = W_RL @ s_prev[:N]  # L->R
    G_R = v_R * ((g_RR + g_RL) + envelope * beta_0) + g_gp * Wgp_R @ p_new

    G = np.vstack([G_L, G_R])
        
    # Linear transfer function (ReLU) + Poisson Spike
    F = G * (G >= 0)  # ReLU activation
    # 修改为确定性形式
    # spk_count = spk_count % m  # Keep remainder 跨时间步骤累计参与的spike量
    spk[:,t:t+1] = F * dt  # 确定性形式
    # Update population activity
    s_new = s_prev + spk[:, t:t+1] - s_prev * dt / tau_s
    r_all[:,t] = s_new.flatten()

    # Hebbian update for place -> grid synapses
    hebb_L = np.outer(s_prev[:N].flatten(), p_new.flatten())
    hebb_R = np.outer(s_prev[N:].flatten(), p_new.flatten())
    # break
    Wgp_L += hebb_lr * hebb_L
    Wgp_R += hebb_lr * hebb_R
    norms_L = np.linalg.norm(Wgp_L, axis=0, keepdims=True)   # axis = 1 对于每一行单独计算，即网格细胞输入归一化 
    norms_R = np.linalg.norm(Wgp_R, axis=0, keepdims=True)    # axis = 0 对于每一列单独计算, 位置细胞归一化.每个位置细胞的输出相同
    norms_L[norms_L == 0] = 1.0    # 避免除0问题
    norms_R[norms_R == 0] = 1.0 
    Wgp_L = Wgp_L / norms_L
    Wgp_R = Wgp_R / norms_R
    
    s_prev = s_new
    p_prev = p_new
   

####  Visualize Learning Process and Results

##### 可视化学习后的Grid和HPC pattern

In [ ]:
%matplotlib inline
grid_pattern = np.mean(r_all[:2*N,-1].reshape(2,N), axis=0)
plt.figure()
plt.plot(grid_pattern)
plt.show()

In [ ]:
fig,ax_place = plt.subplots(1,1, figsize = (10,1))
initial_place = r_place[:, -1]
initial_vmax = max(initial_place.max(), 1e-9)
pattern_im = ax_place.imshow(
    initial_place[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=initial_vmax,
)
ax_place.set_yticks([])
ax_place.set_xlabel('Place cell index')
ax_place.set_title('Hippocampal activity (last timestamp)')
ax_place.set_xlim(0, Nh)

##### 可视化突触权重和网络迭代的过程

可视化Wgp

In [ ]:
# 可视化W_gp
fig, (ax_L, ax_R) = plt.subplots(1, 2, figsize=(14, 5))

# Plot Wgp_L
im_L = ax_L.imshow(Wgp_L, aspect='auto', cmap='viridis')
ax_L.set_xlabel('Place cell index')
ax_L.set_ylabel('Grid cell index (Left)')
ax_L.set_title(f'Wgp_L (sum={Wgp_L.sum():.2f})')
cbar_L = plt.colorbar(im_L, ax=ax_L, orientation='vertical', pad=0.02, fraction=0.046)

# Plot Wgp_R
im_R = ax_R.imshow(Wgp_R, aspect='auto', cmap='viridis')
ax_R.set_xlabel('Place cell index')
ax_R.set_ylabel('Grid cell index (Right)')
ax_R.set_title(f'Wgp_R (sum={Wgp_R.sum():.2f})')
cbar_R = plt.colorbar(im_R, ax=ax_R, orientation='vertical', pad=0.02, fraction=0.046)

# Set global colorbar limits
vmax_global = max(Wgp_L.max(), Wgp_R.max())
vmin_global = min(Wgp_L.min(), Wgp_R.min())
im_L.set_clim(vmin_global, vmax_global)
im_R.set_clim(vmin_global, vmax_global)

plt.tight_layout()
plt.show()


迭代动画

In [ ]:
%matplotlib widget
fig, (ax, ax_place) = plt.subplots(
    2, 1, figsize=(10, 7), gridspec_kw={'height_ratios': [4, 0.8], 'hspace': 0.4}
 )
# Initialize grid-cell activity lines
line_left, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'r-', alpha=0.7, label='Left Population')
line_right, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'b-', alpha=0.7, label='Right Population')
line_combined, = ax.plot(x_prefs, np.zeros_like(x_prefs), 'g-', linewidth=2, label='Combined')
# Add position indicator
# pos_line = ax.axvline(0, color='k', linestyle='--', alpha=0.5, label='Current Position')

ax.set_xlim(0, 1)
ax.set_ylim(0, np.max(r_all) * 2.1)
ax.set_autoscalex_on(False)
ax.set_autoscaley_on(False)
ax.set_xlabel('Position Preference')
ax.set_ylabel('Neural Activity')
# ax.set_title('1D grid Fiete 2009 x = {}')
ax.legend()
ax.grid(True, alpha=0.3)

# Hippocampal activity heatmap (1D vector displayed as a thin strip)
initial_place = r_place[:, 0]
initial_vmax = max(initial_place.max(), 1e-9)
pattern_im = ax_place.imshow(
    initial_place[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=initial_vmax,
)
ax_place.set_yticks([])
ax_place.set_xlabel('Place cell index')
ax_place.set_title('Hippocampal activity (current timestep)')
ax_place.set_xlim(0, Nh)
# 创建垂直colorbar并设置位置
cbar = fig.colorbar(pattern_im, ax=ax_place, orientation='vertical', 
                    pad=0.05, fraction=0.05, shrink=0.8)
cbar.set_label('FR(norm)', rotation=270, labelpad=15)
# 或者使用set_position调整（单位为图形坐标）
# cbar.ax.set_position([0.85, 0.15, 0.02, 0.7])  # [左, 下, 宽, 高]
def update(frame):
    t_idx = frame
        

    # Update activity lines
    left_activity = r_all[:N, t_idx]
    right_activity = r_all[N:, t_idx]
    combined_activity = left_activity + right_activity

    line_left.set_ydata(left_activity)
    line_right.set_ydata(right_activity)
    line_combined.set_ydata(combined_activity)
    # Update title
    ax.set_title(f' t = {dt * t_idx:.4f}s x = {round(x[t_idx], 2)}')
    # Update position line
    # current_pos = x[t_idx]
    # pos_line.set_xdata([current_pos, current_pos])
    
    # Update hippocampal heatmap
    place_activity = r_place[:, t_idx]
    vmax = max(place_activity.max(), 1e-9)
    pattern_im.set_data(place_activity[None, :])
    pattern_im.set_clim(0, vmax)
    cbar.update_normal(pattern_im)

    return line_left, line_right, line_combined, pattern_im

# Create animation
frame_step = 20
ani = FuncAnimation(
    fig,
    update,
    frames=range(0, min(n_steps, 20000), frame_step),  # Limit to 1000 frames for performance
    interval=50,
    blit=True,
    repeat=True,
 )
plt.show()
plt.tight_layout()


In [ ]:
ani.event_source.stop()
%matplotlib inline

In [ ]:
print("Saving animation...")
folder = 'visualization'
cur_dir = os.getcwd()
folder_path = os.path.join(cur_dir, folder)
print(folder_path)
if not os.path.exists(folder_path):
    print(f'文件夹{folder}生成')
    os.makedirs(folder_path)
timestamp = datetime.now().strftime("%m%d_%H%M%S")
# ani.save('neural_sheet_activity'+timestamp+'.mp4', writer='ffmpeg', fps=20, dpi=100)
ani.save(os.path.join(folder_path, f'place-grid_1M1D_{N}'+timestamp+'.gif'), writer='pillow', fps=20, dpi=100)
print("Animation saved")

### 3. 吸引子验证
+ 提取grid_org（s_prev）左右环平均, 为
+ 从grid pattern投射到p，得到p_org
+ 加入高斯噪声，从p_org得到p_noise
+ 从p_noise开始更新，观察网络输出g_output是否会收敛回到grid_org

In [ ]:
from numpy import dot
from numpy.linalg import norm
def euclidean_similarity(v1, v2):
    dist = np.linalg.norm(v1 - v2)
    # 将距离转换为相似度（0-1范围）
    similarity = 1 / (1 + dist)
    return similarity
def cosine_similarity(v1, v2):
    """
    计算两个向量的余弦相似度
    范围：[-1, 1]，1表示完全相同，0表示正交，-1表示完全相反
    """
    cos_sim = dot(v1, v2) / (norm(v1) * norm(v2))
    return cos_sim

In [ ]:
grid_org = np.mean(r_all[:2*N,-1].reshape(2,N), axis=0)
p_org = r_place[:, -1]

In [ ]:
def g_p_denoise(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = 2, dt = 0.01, noise_std = 0.2, sim_measure = 'Cosine'):
    '''验证学习后的网络的去噪能力
    返回噪声后的初始位置细胞状态p_noise，最终位置细胞状态p_test，(最后)位置细胞相似度sim
    ----------  
   '''
    # place cell initial state with noise_std
    p_noise = p_org + np.random.normal(0, noise_std, p_org.shape)
    p_noise = np.maximum(p_noise, 0)  # ensure non-negative
    
    # r_test = np.zeros((len(grid_org)*2, 1))

    # grid的初始状态
    # s_test = np.hstack([grid_org, grid_org]).T.reshape(-1, 1)  # (128, 1)
    s_test = np.zeros((len(grid_org)*2,1))
    n_steps_test = int(T_max/dt)
    v_test = np.zeros(n_steps_test)  # zero velocity for attractor test


    # s_test为前一时刻的grid活动
    t = 1
    sim = 0.0
    p_test = p_noise.copy()
    while (sim < 0.99) & (t < n_steps_test - 1):
        # Grid cell dynamics (no learning, fixed weights)
        v_L = 1 - e_mu * v_test[t]
        v_R = 1 + e_mu * v_test[t]
        
        g_LL = W_LL @ s_test[:N]
        g_LR = W_LR @ s_test[N:]
        g_RR = W_RR @ s_test[N:]
        g_RL = W_RL @ s_test[:N]
    
        G_L = v_L * ((g_LL + g_LR) + envelope * beta_0) + g_gp * Wgp_L @ p_test.reshape(-1,1)
        G_R = v_R * ((g_RR + g_RL) + envelope * beta_0) + g_gp * Wgp_R @ p_test.reshape(-1,1)
        
        G = np.vstack([G_L, G_R])
        F = G * (G >= 0)
        
        s_new = s_test + F * dt - s_test * dt / tau_s
        # grid activity更新并储存
        # r_test[:, t] = s_new.flatten()
        s_test = s_new
        
        # Place cell dynamics
        hippo_drive = gain_pg * W_pg @ s_new + Iext2
        phi_term = relu(hippo_drive - theta).reshape(-1, 1)
        p_new = p_test + dt / tau_h * (-p_test + phi_term.flatten())
        # r_place_test[:, t:t+1] = p_new.reshape(-1, 1)
        p_test = p_new.flatten()
        if sim_measure == 'Eucliean':
            sim = euclidean_similarity(p_new, p_org)
        elif sim_measure == 'Cosine':
            sim = cosine_similarity(p_new, p_org)
        t += 1
    return p_noise, p_test, sim

In [ ]:
def g_p_denoise_process(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = 2, dt = 0.01, noise_std = 0.2, sim_measure = 'Cosine'):
    '''验证学习后的网络的去噪能力
    在g_p_denoise()的基础上返回每一时刻的sim
    p_noise: 去噪声前
    p_test: 去噪声后
    sim_arr: 相似度数组 shape = (timestamps, 1)
    '''
    # place cell initial state with noise_std
    p_noise = p_org + np.random.normal(0, noise_std, p_org.shape)
    p_noise = np.maximum(p_noise, 0)  # ensure non-negative

    # grid的初始状态
    # s_test = np.hstack([grid_org, grid_org]).T.reshape(-1, 1)  # (128, 1)
    s_test = np.zeros((len(grid_org)*2,1))
    n_steps_test = int(T_max/dt)
    v_test = np.zeros(n_steps_test)  # zero velocity for attractor test


    # s_test为前一时刻的grid活动
    t = 1
    sim = 0.0
    sim_arr = np.zeros(int(T_max/dt))
    p_test = p_noise.copy()
    while (t < n_steps_test):
        # Grid cell dynamics (no learning, fixed weights)
        v_L = 1 - e_mu * v_test[t]
        v_R = 1 + e_mu * v_test[t]
        
        g_LL = W_LL @ s_test[:N]
        g_LR = W_LR @ s_test[N:]
        g_RR = W_RR @ s_test[N:]
        g_RL = W_RL @ s_test[:N]
    
        G_L = v_L * ((g_LL + g_LR) + envelope * beta_0) + g_gp * Wgp_L @ p_test.reshape(-1,1)
        G_R = v_R * ((g_RR + g_RL) + envelope * beta_0) + g_gp * Wgp_R @ p_test.reshape(-1,1)
        
        G = np.vstack([G_L, G_R])
        F = G * (G >= 0)
        
        s_new = s_test + F * dt - s_test * dt / tau_s
        # grid activity更新并储存
        # r_test[:, t] = s_new.flatten()
        s_test = s_new
        
        # Place cell dynamics
        hippo_drive = gain_pg * W_pg @ s_new + Iext2
        phi_term = relu(hippo_drive - theta).reshape(-1, 1)
        p_new = p_test + dt / tau_h * (-p_test + phi_term.flatten())
        # r_place_test[:, t:t+1] = p_new.reshape(-1, 1)
        p_test = p_new.flatten()
        if sim_measure == 'Eucliean':
            sim = euclidean_similarity(p_new, p_org)
        elif sim_measure == 'Cosine':
            sim = cosine_similarity(p_new, p_org)
        sim_arr[t] = sim
        t += 1
    return p_noise, p_test, sim_arr

In [ ]:
def attractor_test2(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = 2, dt = 0.01, Nrun = 100,noise_stds = [2], sim_measure = 'Cosine'):
    Nn = len(noise_stds)
    n_steps_test = int(T_max / dt)
    p_sim_all = np.zeros((Nrun, Nn, n_steps_test))  # collect p similarity over time_steps
    for n in range(Nn):
        noise_std = noise_stds[n]
        for r in range(Nrun):
            p_noise, p_denoised, sim_array = g_p_denoise_process(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max, dt, noise_std, sim_measure)
            p_sim_all[r, n, :] = sim_array
    
    return p_sim_all

In [ ]:
def attractor_test(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = 2, dt = 0.01, Nrun = 100,noise_stds = [2], sim_measure = 'Cosine'):
    Nn = len(noise_stds)
    n_steps_test = int(T_max / dt)
    p_sim_all = np.zeros((Nrun, Nn, n_steps_test))  # collect p similarity over time_steps
    for n in range(Nn):
        noise_std = noise_stds[n]
        for r in range(Nrun):

            # place cell initial state with noise_std
            p_noise = p_org + np.random.normal(0, noise_std, p_org.shape)
            p_noise = np.maximum(p_noise, 0)  # ensure non-negative
            
            # r_test = np.zeros((len(grid_org)*2, 1))

            # grid的初始状态
            # s_test = np.hstack([grid_org, grid_org]).T.reshape(-1, 1)  # (128, 1)
            s_test = np.zeros((len(grid_org)*2,1))
            
            v_test = np.zeros(n_steps_test)  # zero velocity for attractor test


            # s_test为前一时刻的grid活动
            t = 1
            sim = 0.0
            p_test = p_noise.copy()
            while (sim < 0.99) & (t < n_steps_test - 1):
                # Grid cell dynamics (no learning, fixed weights)
                v_L = 1 - e_mu * v_test[t]
                v_R = 1 + e_mu * v_test[t]
                
                g_LL = W_LL @ s_test[:N]
                g_LR = W_LR @ s_test[N:]
                g_RR = W_RR @ s_test[N:]
                g_RL = W_RL @ s_test[:N]
            
                G_L = v_L * ((g_LL + g_LR) + envelope * beta_0) + g_gp * Wgp_L @ p_test.reshape(-1,1)
                G_R = v_R * ((g_RR + g_RL) + envelope * beta_0) + g_gp * Wgp_R @ p_test.reshape(-1,1)
                
                G = np.vstack([G_L, G_R])
                F = G * (G >= 0)
                
                s_new = s_test + F * dt - s_test * dt / tau_s
                # grid activity更新并储存
                # r_test[:, t] = s_new.flatten()
                s_test = s_new
                
                # Place cell dynamics
                hippo_drive = gain_pg * W_pg @ s_new + Iext2
                phi_term = relu(hippo_drive - theta).reshape(-1, 1)
                p_new = p_test + dt / tau_h * (-p_test + phi_term.flatten())
                # r_place_test[:, t:t+1] = p_new.reshape(-1, 1)
                p_test = p_new.flatten()
                if sim_measure == 'Eucliean':
                    sim = euclidean_similarity(p_new, p_org)
                elif sim_measure == 'Cosine':
                    sim = cosine_similarity(p_new, p_org)
                p_sim_all[r, n, t] = sim
                t += 1
    return p_sim_all

In [ ]:
ls_noise = [0,2,4,8,10,20,40]
T_max = 1.5
dt = 0.01
p_sim = attractor_test2(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = T_max, dt = dt,
                        Nrun = 100,noise_stds = ls_noise, sim_measure = 'Cosine')

# plot the results
plt.figure(figsize=(10, 6))
plt.grid(True, color = 'grey', ls = 'dashed', lw = 2, alpha=0.3)
for i, noise_std in enumerate(ls_noise):
    mean_sim = np.mean(p_sim[:, i, :], axis=0)
    std_sim = np.std(p_sim[:, i, :], axis=0)
    # plt.errorbar(np.arange(0,T_max, dt), mean_sim, yerr=std_sim, label=f'Noise STD={noise_std}')
    plt.fill_between(np.arange(0, T_max, dt), mean_sim - std_sim, mean_sim + std_sim, alpha=0.15)
    plt.plot(np.arange(0, T_max, dt), mean_sim, label=f'Noise STD={noise_std}', linewidth=2)
plt.ylim(0,1.1)

plt.xlabel('Time step')
plt.ylabel('Similarity')
plt.title('Attractor Test: Place Cell Similarity Over Time')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# test
n_std = 5
t_max = 2
p_noise, p_final, sim = g_p_denoise(grid_org, p_org, Wgp_L, Wgp_R, W_pg, T_max = t_max, 
                                    dt = 0.01, noise_std = n_std,
                                    sim_measure = 'Cosine')

# visualization of original and noisy place cell activity
fig, ax_place_noise = plt.subplots(3,1, figsize = (10,3))

noise_vmax = max(p_noise.max(), 1e-9)   
fig.suptitle(f'Denoising{T_max}s, noise_std={n_std}, final_sim={sim:.3}')
pattern_im_org = ax_place_noise[0].imshow(
    p_org[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=noise_vmax,
)
pattern_im_noise = ax_place_noise[1].imshow(
    p_noise[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=noise_vmax,
)
pattern_im_final = ax_place_noise[2].imshow(
    p_final[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=noise_vmax,
)

ax_place_noise[0].set_yticks([])
# ax_place_noise[0].set_xlabel('Place cell index')
ax_place_noise[0].set_title('Original hippocampal activity (last timestamp)')
ax_place_noise[0].set_xlim(0, Nh)
ax_place_noise[1].set_yticks([])
# ax_place_noise[1].set_xlabel('Place cell index')   
ax_place_noise[1].set_title('Noisy hippocampal activity')
ax_place_noise[1].set_xlim(0, Nh)
ax_place_noise[2].set_yticks([])
ax_place_noise[2].set_xlabel('Place cell index')   
ax_place_noise[2].set_title('Final hippocampal activity')
ax_place_noise[2].set_xlim(0, Nh)
plt.tight_layout()
plt.show()


##### Visualize denoising process in animation

噪声从p加入，从p_org变成p_noise  
grid cell的初始状态：  
1. 初始活动为0，从只接收p的投射开始演化
2. 初始活动为g_org (即无噪声前的)

In [ ]:
%matplotlib widget
fig = plt.figure(figsize=(12, 8))
gs = fig.add_gridspec(3, 1, height_ratios=[3, 0.8, 0.8], hspace=0.4)
ax_grid = fig.add_subplot(gs[0])
ax_place_org = fig.add_subplot(gs[1])
ax_place_test = fig.add_subplot(gs[2])
s_org_mean =  s_org              # np.mean(s_org.reshape(2,-1), axis=0)
# Initialize grid cell plot
line_org, = ax_grid.plot(x_prefs.flatten(), s_org_mean.flatten(), 'gray', linestyle='--', linewidth=2, alpha=0.7, label='s_org')
line_current, = ax_grid.plot(x_prefs.flatten(), np.zeros(N), 'b-', linewidth=2, label='Current grid pattern')
ax_grid.set_xlim(0, 1)
ax_grid.set_ylim(0, np.max(s_org) * 1.2)
ax_grid.set_xlabel('Position Preference')
ax_grid.set_ylabel('Neural Activity')
ax_grid.set_title('Grid Cell Denoising')
ax_grid.legend()
ax_grid.grid(True, alpha=0.3)

# Initialize original place cell heatmap
vmax_place = max(p_org.max(), 1e-9)
im_org = ax_place_org.imshow(
    p_org[None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=vmax_place,
)
ax_place_org.set_yticks([])
ax_place_org.set_ylabel('Original')
ax_place_org.set_xlim(0, Nh)
cbar_org = plt.colorbar(im_org, ax=ax_place_org, orientation='vertical', pad=0.02, fraction=0.046)
cbar_org.set_label('FR', rotation=270, labelpad=15)

# Initialize current place cell heatmap
im_test = ax_place_test.imshow(
    p_test[:, 0][None, :],
    aspect='auto',
    cmap='gray',
    extent=[0, Nh, 0, 1],
    vmin=0,
    vmax=vmax_place,
)
ax_place_test.set_yticks([])
ax_place_test.set_xlabel('Place Cell Index')
ax_place_test.set_ylabel('Current')
ax_place_test.set_xlim(0, Nh)
cbar_test = plt.colorbar(im_test, ax=ax_place_test, orientation='vertical', pad=0.02, fraction=0.046)
cbar_test.set_label('FR', rotation=270, labelpad=15)

def update(frame):
    t_idx = frame
    
    # Update grid cell pattern
    grid_current = np.mean(r_test[:, t_idx].reshape(2, N), axis=0)
    line_current.set_ydata(grid_current)
    ax_grid.set_title(f'Grid started from Zero activity (t = {dt * t_idx:.4f}s)')
    
    # Update current place cell activity
    place_current = p_test[:, t_idx]
    im_test.set_data(place_current[None, :])
    vmax_current = max(place_current.max(), 1e-9)
    im_test.set_clim(0, vmax_current)
    cbar_test.update_normal(im_test)
    
    return line_current, im_test

# ani = FuncAnimation(
#     fig,
#     update,
#     frames=range(n_steps_test),
#     interval=50,
#     blit=True,
#     repeat=True,
# )
# plt.show()
# Save animation with frame skipping for smoother playback
skip = 100  # display every 10th frame
ani = FuncAnimation(
    fig,
    update,
    frames=range(0, n_steps_test, skip),
    interval=50,
    blit=True,
    repeat=True,
)
plt.show()

In [ ]:
ani.event_source.stop()
%matplotlib inline


In [ ]:

print("Saving animation...")
folder = 'visualization'
cur_dir = os.getcwd()
folder_path = os.path.join(cur_dir, folder)
print(folder_path)
if not os.path.exists(folder_path):
    print(f'文件夹{folder}生成')
    os.makedirs(folder_path)
timestamp = datetime.now().strftime("%m%d_%H%M%S")
# ani.save('neural_sheet_activity'+timestamp+'.mp4', writer='ffmpeg', fps=20, dpi=100)
ani.save(os.path.join(folder_path, f'grid_dnoise2_1M1D_{N}'+timestamp+'.gif'), writer='pillow', fps=20, dpi=100)
print("Animation saved")